In [12]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier


## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [13]:
## Removing the observations with N/As
heart = heart.dropna()

In [14]:
## Defining the input and target variables
X = heart[['age','totChol', 'sysBP', 'BMI','heartRate', 'glucose']]
Y = heart['TenYearCHD']

## SPlitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [15]:
## Building the AdaBoost Model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.001).fit(X_train, Y_train)

## Predicting on test
ada_pred = ada_md.predict_proba(X_test)[:,1]

## Changing likelihoods to labels
ada_labels = np.where(ada_pred < 0.1, 0, 1)

## Computing the recall score
ada_recall = recall_score(Y_test, ada_labels)
print('The recall score for the Ada Boost model is', ada_recall)

The recall score for the Ada Boost model is 0.9017857142857143


In [19]:
## Builing the GradientBoost Model
gb_md = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.001).fit(X_train, Y_train)

## Predicting on test
gb_pred = gb_md.predict_proba(X_test)[:,1]

## Changing likelihoods to labels
gb_labels = np.where(gb_pred < 0.1, 0, 1)

## Computing the recall score
gb_recall = recall_score(Y_test, gb_labels)
print('The recall score for the Gradient Boost model is', gb_recall)

The recall score for the Gradient Boost model is 1.0


In [ ]:
## Based on the data, the best model to use would be the Gradient Boost Model, there is a larger recall score.